In [3]:
import pandas as pd
import chardet

In [7]:
with open("dataset_1382851.csv", "rb") as f:
    result = chardet.detect(f.read(100000))
    encoding = result["encoding"]

print(f"Detected encoding: {encoding}")

df = pd.read_csv("dataset_1382851.csv", encoding=encoding, delimiter=";")


Detected encoding: ISO-8859-1


In [9]:
df

,s_696_1,s_696_2,s_698,s_25,s_132,s_133,s_134,s_919,s_700,s_135,...,v_4,v_5,v_6,v_7,email,statoverall_1,statoverall_2,statoverall_3,statoverall_4,statoverall_5
0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,14~,"Høgevej 8, 6705 Esbjerg Ø",ses@business.aau.dk,0,0,0,1,0
1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,...,1,1,1-5~,"Østre Stationsvej 41P, 5000 Odense C",ses@business.aau.dk,0,0,0,1,0
2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,1,Vognmænd og speditører med 6+ biler,...,1,1,2~,"Store Torv 17 1, 3700 Rønne",ses@business.aau.dk,0,0,0,1,0
3,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,1,"Kjellerupsgade 24, 9000 Aalborg",ses@business.aau.dk,0,0,0,1,0
4,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,1~,"Traps Allé 8, 2500 Valby",ses@business.aau.dk,0,0,0,1,0
5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,2,1,NaN,"Hallindskovvej 60, 5700 Svendborg",ses@business.aau.dk,0,0,0,1,0
6,0,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,...,1,1,1-5~,"Brandevej 10 dør 11, 9220 Aalborg Øst",ses@business.aau.dk,0,0,0,1,0
7,1,1,1.0,"Sejlbådsejere der vægter sikkerhed, innovation...",NaN,NaN,NaN,NaN,1,Bådeværfter (Sejlbåde og yachtbyggere),...,1,1,NaN,"Kirkerupvej 39, 4000 Roskilde",ses@business.aau.dk,0,0,0,1,0


In [ ]:
# main.py
#!/usr/bin/env python
import yaml
from typing import Dict, Optional
from pydantic import BaseModel
from crewai import LLM
from crewai.flow.flow import Flow, listen, start, router
from business_plan_creator.crews.businessplan_crew.businessplan_crew import BusinessPlanCrew
from business_plan_creator.crews.review_business_plan.review_business_plan import ReviewBusinessPlanCrew
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

class BusinessPlanState(BaseModel):
    business_concept: str = ""
    market_input: str = ""
    competition_input: str = ""
    product_input: str = ""
    marketing_input: str = ""
    financial_input: str = ""
    business_plan_content: str = ""
    feedback: Optional[str] = None
    valid: bool = False
    retry_count: int = 0

class BusinessPlanFlow(Flow[BusinessPlanState]):

    @start()
    def get_user_input(self):
        """Get input from the user about the business concept and additional considerations"""
        print("\n=== Create Your Business Plan ===\n")
        self.state.business_concept = input("What business concept would you like to create a business plan for? ")
        print("\nAdditional Considerations:")
        self.state.market_input = input("Enter any market considerations (optional, press Enter to skip): ")
        self.state.competition_input = input("Enter any competition considerations (optional, press Enter to skip): ")
        self.state.product_input = input("Enter any product considerations (optional, press Enter to skip): ")
        self.state.marketing_input = input("Enter any marketing considerations (optional, press Enter to skip): ")
        self.state.financial_input = input("Enter any financial considerations (optional, press Enter to skip): ")
        print(f"\nCreating a business plan\n")
        return self.state

    def run_with_api_input(self, state: BusinessPlanState):
        """Run the flow with input from the API"""
        self.state = state
        return self.kickoff()
    
    @listen(get_user_input)
    @listen("retry")
    def create_business_plan(self, state: BusinessPlanState):
        """Create the business plan using the crew with Gemini LLM"""
        print("Creating business plan...")
        
        # Initialize the LLM with Gemini
        llm = LLM(
            model="gemini/gemini-2.0-flash",
            temperature=0.7
        )

        # Load configurations
        with open("src/business_plan_creator/crews/businessplan_crew/config/agents.yaml", "r") as f:
            agents_config = yaml.safe_load(f)
        
        with open("src/business_plan_creator/crews/businessplan_crew/config/tasks.yaml", "r") as f:
            tasks_config = yaml.safe_load(f)

        # Create and run the crew
        crew = BusinessPlanCrew(agents_config, tasks_config, llm)
        result = crew.crew.kickoff(inputs={
            "business_concept": state.business_concept,
            "market_input": state.market_input,
            "competition_input": state.competition_input,
            "product_input": state.product_input,
            "marketing_input": state.marketing_input,
            "financial_input": state.financial_input,
            "feedback": state.feedback
        })
        
        self.state.business_plan_content = result.raw

        print("\nBusiness plan creation completed successfully")
        return "Business plan created"
    
    @router(create_business_plan)
    def review_business_plan(self):
        if self.state.retry_count > 3:
            return "max_retry_exceeded"
        
        llm = LLM(
            model="gemini/gemini-2.0-flash",
            temperature=0.7
        )

        with open("src/business_plan_creator/crews/review_business_plan/config/agents.yaml", "r") as f:
            agents_config = yaml.safe_load(f)
        
        with open("src/business_plan_creator/crews/review_business_plan/config/tasks.yaml", "r") as f:
            tasks_config = yaml.safe_load(f)

        crew = ReviewBusinessPlanCrew(agents_config, tasks_config, llm)
        result = crew.crew().kickoff(inputs={"business_plan_content": self.state.business_plan_content})
        self.state.valid = result["valid"]
        self.state.feedback = result["feedback"]
        self.state.retry_count += 1
        
        if self.state.valid:
            return "completed"
        
        print("RETRY")
        return "retry"

    @listen("completed")
    def save_result(self):
        """Return the final business plan content"""
        return {
            "business_plan": self.state.business_plan_content,
            "status": "completed"
        }

    @listen("max_retry_exceeded")
    def max_retry_exceeded_exit(self):
        print("Max retries exceeded and quitting")


def kickoff():
    """Run the business plan creator flow"""
    BusinessPlanFlow().kickoff()
    print("\n=== Flow Complete ===")


def plot():
    """Generate a visualization of the flow"""
    flow = BusinessPlanFlow()
    flow.plot("business_plan_creator_flow")
    print("Flow visualization saved to business_plan_creator_flow.html")


if __name__ == "__main__":
    kickoff()


In [ ]:
# backend.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from crewai import LLM
import yaml
from .crews.businessplan_crew.businessplan_crew import BusinessPlanCrew
from main import BusinessPlanFlow, BusinessPlanState
from typing import List, Dict, Optional
import os
import google.generativeai as genai

app = FastAPI()

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

class BusinessPlanRequest(BaseModel):
    business_concept: str
    market_input: str
    competition_input: str
    product_input: str
    marketing_input: str
    financial_input: str

class AgentResponse(BaseModel):
    role: str
    content: str

class BusinessPlanResponse(BaseModel):
    business_plan: str
    status: str
    feedback: Optional[str] = None
    valid: bool = False

@app.post("/generate-business-plan", response_model=BusinessPlanResponse)
async def generate_business_plan(request: BusinessPlanRequest):
    try:
        # Create a new BusinessPlanState with the request data
        state = BusinessPlanState(
            business_concept=request.business_concept,
            market_input=request.market_input,
            competition_input=request.competition_input,
            product_input=request.product_input,
            marketing_input=request.marketing_input,
            financial_input=request.financial_input
        )
        
        # Create and run the flow with API input
        flow = BusinessPlanFlow()
        flow.run_with_api_input(state)
        
        # Get the final state after the flow completes
        final_state = flow.state
        
        return BusinessPlanResponse(
            business_plan=final_state.business_plan_content,
            status="completed" if final_state.valid else "failed",
            feedback=final_state.feedback,
            valid=final_state.valid
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e)) 

In [ ]:
# business_plan_crew.py
from crewai import Agent, Crew, Task, Process

class BusinessPlanCrew:
    """Business Plan Crew"""

    def __init__(self, agents_config, tasks_config, llm):
        self.agents_config = agents_config
        self.tasks_config = tasks_config
        self.llm = llm
        self.agents = self.create_agents()
        self.tasks = self.create_tasks()
        self.crew = self.create_crew()

    def create_agents(self):
        business_designer = Agent(
            role=self.agents_config["business_designer"]["role"],
            goal=self.agents_config["business_designer"]["goal"],
            backstory=self.agents_config["business_designer"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        market_analyst = Agent(
            role=self.agents_config["market_analyst"]["role"],
            goal=self.agents_config["market_analyst"]["goal"],
            backstory=self.agents_config["market_analyst"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        competition_analyst = Agent(
            role=self.agents_config["competition_analyst"]["role"],
            goal=self.agents_config["competition_analyst"]["goal"],
            backstory=self.agents_config["competition_analyst"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        product_designer = Agent(
            role=self.agents_config["product_designer"]["role"],
            goal=self.agents_config["product_designer"]["goal"],
            backstory=self.agents_config["product_designer"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        marketing_expert = Agent(
            role=self.agents_config["marketing_expert"]["role"],
            goal=self.agents_config["marketing_expert"]["goal"],
            backstory=self.agents_config["marketing_expert"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        operations_specialist = Agent(
            role=self.agents_config["operations_specialist"]["role"],
            goal=self.agents_config["operations_specialist"]["goal"],
            backstory=self.agents_config["operations_specialist"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        financial_expert = Agent(
            role=self.agents_config["financial_expert"]["role"],
            goal=self.agents_config["financial_expert"]["goal"],
            backstory=self.agents_config["financial_expert"]["backstory"],
            verbose=True,
            llm=self.llm
        )

        return [business_designer, market_analyst, competition_analyst, product_designer, marketing_expert, operations_specialist, financial_expert]

    def create_tasks(self):
        create_business_concept = Task(
            description=self.tasks_config["create_business_concept"]["description"],
            expected_output=self.tasks_config["create_business_concept"]["expected_output"],
            agent=self.agents[0]
        )

        create_market_analysis = Task(
            description=self.tasks_config["create_market_analysis"]["description"],
            expected_output=self.tasks_config["create_market_analysis"]["expected_output"],
            agent=self.agents[1],
            context=[create_business_concept]
        )

        create_competition_analysis = Task(
            description=self.tasks_config["create_competition_analysis"]["description"],
            expected_output=self.tasks_config["create_competition_analysis"]["expected_output"],
            agent=self.agents[2],
            context=[create_business_concept, create_market_analysis]
        )

        create_product_design = Task(
            description=self.tasks_config["create_product_design"]["description"],
            expected_output=self.tasks_config["create_product_design"]["expected_output"],
            agent=self.agents[3],
            context=[create_business_concept, create_market_analysis, create_competition_analysis]
        )

        create_marketing_plan = Task(
            description=self.tasks_config["create_marketing_plan"]["description"],
            expected_output=self.tasks_config["create_marketing_plan"]["expected_output"],
            agent=self.agents[4],
            context=[create_business_concept, create_market_analysis, create_competition_analysis, create_product_design] 
        )

        create_operating_plan = Task(
            description=self.tasks_config["create_operating_plan"]["description"],
            expected_output=self.tasks_config["create_operating_plan"]["expected_output"],
            agent=self.agents[5],
            context=[create_business_concept, create_market_analysis, create_competition_analysis, create_product_design, create_marketing_plan]
        )

        create_financial_plan = Task(
            description=self.tasks_config["create_financial_plan"]["description"],
            expected_output=self.tasks_config["create_financial_plan"]["expected_output"],
            agent=self.agents[6],
            context=[create_business_concept, create_market_analysis, create_competition_analysis, create_product_design, create_marketing_plan, create_operating_plan]
        )

        return [create_business_concept, create_market_analysis, create_competition_analysis, create_product_design, create_marketing_plan, create_operating_plan, create_financial_plan]

    def create_crew(self):
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True
        )
